In [ ]:
"""
%pip install matplotlib==3.10.1
%pip install pennylane==0.42.3
%pip install numpy==1.26.4
%pip install pandas==2.2.2
%pip install basis-set-exchange
%pip install qiskit-aer==0.17.1
%pip install pennylane-qiskit==0.40.1
%pip install qiskit-aer-gpu==0.15.1
"""

'\n%pip install matplotlib==3.10.1\n%pip install pennylane==0.42.3\n%pip install numpy==1.26.4\n%pip install pandas==2.2.2\n%pip install basis-set-exchange\n%pip install qiskit-aer==0.17.1\n%pip install pennylane-qiskit==0.40.1\n%pip install qiskit-aer-gpu==0.15.1\n'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 1º: Create the Molecule and Hamiltonian

In the code cells below, we create the molecular geometry of H2 with the radial distance of 1,401 a.u. with the optimal internuclear distance of Szabo and Ostlund. We will use the STO-3G minimal basis set by importing the parameters of the basis set exchange functions. Pennylane allows us to construct the Hamiltonian from the second quantization with the Jordan-Wigner transformation.

In [ ]:
from pennylane import AdamOptimizer
from qiskit_aer import AerSimulator
from pennylane import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
import time
import json

symbols = ["H", "H"]
coordinates = np.array([[0.0, 0.0, 0], [0.0, 0.0, 1.401]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='STO-3G', load_data=True, unit='bohr')
H, qubits = qml.qchem.molecular_hamiltonian(molecule)

print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is  -0.09883485860187924 * I([0, 1, 2, 3]) + 0.17120123803197834 * Z(0) + 0.17120123803197845 * Z(1) + 0.16862327620358059 * (Z(0) @ Z(1)) + -0.22279639115527738 * Z(2) + 0.12054612718324412 * (Z(0) @ Z(2)) + 0.16586801098505832 * (Z(1) @ Z(2)) + 0.045321883801814206 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.045321883801814206 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.045321883801814206 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.045321883801814206 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.22279639115527738 * Z(3) + 0.16586801098505832 * (Z(0) @ Z(3)) + 0.12054612718324412 * (Z(1) @ Z(3)) + 0.17434948668373768 * (Z(2) @ Z(3))


# 2º: Calculate the Number of Shots Necessary for the Experiment

In the code cells below, we calculate the number of shots required for our noise-free experiment using the ADAM optimizer. This calculation is based on sampling formulas and a specified precision, denoted by 𝜖. After determining the required number of shots, we round it to 4,000,000. Increasing the precision will result in longer computation times and may also require more memory.

In [ ]:
e = 1e-3
cj = H.terms()[0]

sum_squarred = sum([abs(float(c)) for c in cj])**2

shots_necessary = round(1/(e**2)*(sum_squarred))
shots_necessary

3935933

In [ ]:
shots_necessary=4000000

# 3º: Configure the Simulator

In the cells below, we configure a noise-free Qiskit Aer simulator using the PennyLane-Qiskit plugin. Our goal is to simulate the UCCSD ansatz with PennyLane. The quantum circuit is designed to compute the expected value of the Hamiltonian by collecting a large number of samples obtained from the shots. For these simulations, we will leverage the GPU to accelerate the computations.

In [ ]:
aer_simulator = AerSimulator(method='statevector', device='GPU')

In [ ]:
dev = qml.device("qiskit.aer", backend=aer_simulator, wires=qubits, shots=shots_necessary)

In [ ]:
electrons = 2
wires = range(qubits)
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qml.qchem.excitations(electrons, qubits)
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

In [ ]:
@qml.qnode(dev, interface="autograd", diff_method="parameter-shift")
def circuit(params):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)

# 4º: Configure the optimizer and run the circuit

In the code cells below, we configure the ADAM optimizer. It is set to a maximum of 250 iterations with a tolerance of 1×10^−8.We configured the hyperparameters according to the seminal ADAM article. We also provide the initial parameter angles to be optimized. The circuit is executed iteratively until convergence is achieved.

In [ ]:
alpha = 0.001
beta1 = 0.9
beta2 = 0.999
eps = 1e-08
steps = 250
conv_tol=1e-8

In [ ]:
optimizer = AdamOptimizer(alpha, beta1, beta2, eps)
current_params = np.zeros(len(singles) + len(doubles))

In [ ]:
energy_data = []
params_data = []

t0 = time.perf_counter()
for step in range(0, steps):
  current_params, current_energy = optimizer.step_and_cost(circuit, current_params)
  energy_data.append(current_energy)
  params_data.append(current_params)

  if len(energy_data) > 1 and abs(energy_data[-1] - energy_data[-2]) < conv_tol:
    t1 = time.perf_counter()
    print(f"\nFinal Energy Estimate: {current_energy:.8f}")
    print(f"Optimal Parameter: {current_params}", )
    print(f"Execution Time: {t1-t0}")
    break

  else:
    print(f"Step {step+1}: Energy = {current_energy:.8f} Ha | θ = {current_params} rad")

else:
    t1 = time.perf_counter()
    print(f"\nStopped at max steps. Last E = {energy_data[-1]:.8f} Ha")
    print(f"Params: {current_params}")
    print(f"Execution Time: {t1 - t0:.4f}s")

Step 1: Energy = -1.11680347 Ha | θ = [0.00099861 0.00099711 0.001     ] rad
Step 2: Energy = -1.11687924 Ha | θ = [0.00057591 0.00039948 0.00199981] rad
Step 3: Energy = -1.11701740 Ha | θ = [-7.42857780e-05 -1.66670270e-04  2.99952029e-03] rad
Step 4: Energy = -1.11717158 Ha | θ = [-0.00049917 -0.00068675  0.00399896] rad
Step 5: Energy = -1.11739926 Ha | θ = [-0.00085771 -0.00081907  0.00499806] rad
Step 6: Energy = -1.11758729 Ha | θ = [-0.00082443 -0.00073373  0.00599679] rad
Step 7: Energy = -1.11778507 Ha | θ = [-0.0006468  -0.00039061  0.00699498] rad
Step 8: Energy = -1.11793213 Ha | θ = [-3.68927497e-04  5.09951392e-05  7.99265276e-03] rad
Step 9: Energy = -1.11810443 Ha | θ = [-0.00021562  0.00044139  0.0089897 ] rad
Step 10: Energy = -1.11828932 Ha | θ = [-5.82552329e-05  6.43501908e-04  9.98599175e-03] rad
Step 11: Energy = -1.11843992 Ha | θ = [-4.13395441e-05  7.17470102e-04  1.09815041e-02] rad
Step 12: Energy = -1.11858611 Ha | θ = [-1.71812399e-05  6.79504929e-04  1.1

# 5º: Save the Data for Later Analysis

In [ ]:
df = pd.DataFrame({
    "energies": energy_data,
    "params": params_data
})

df["energies_json"] = df["energies"].apply(lambda arr: json.dumps(arr.tolist()))
df["params_json"]  = df["params"].apply(lambda arr: json.dumps(arr.tolist()))

In [ ]:
df.to_csv("ADAM_NOISE_FREE.csv", columns=["energies_json","params_json"], index=False)